The 600 first epochs were performed on the reduce dataset (360 000 images). Then to the complete dataset (560 000 images).

With batch size of 640 and 360 000 images in the database, 100 epochs took about 3h (a bit less)
With batch size of 700 and 560 000 images in the database, 100 epochs took about 4h40


WITH METRICS: 200 epochs: Wall time: 11h 33min 22s

Current script load the 'final' weigths (which is save if everything goes well)

Weights are saved every 100 epochs. I should probably keep wieghts each 1000 epochs (depending on how many time it took to treat the whole data set) 139298/139937


Look at the correlation matrix to see if there is a bias in the angle correct prediction. If yes, rebuild the dataset by applying angle of 90/180/270 to try to get something mor or less homogenous
1 step: realign every centriole in a 0->90 orientation

2ns step: perform reorientation. => rotation of 90/180/270 i can also flip the image

Load the 500 epochs save if problem



TO DO:

To check: Note: in train definition i put the model in train mode (model.train() ) I do not change this statut after.

Save the log of accuracy and loss

Save the train_loader and validation_loader to don't have to rebuild them (especially because that lead to mixing each time train and validation set


IMPORTANT: CHANGE PERFORMED ON gestion of classification/Regression problem (modification of n_classes/n_class call in data_agregator and VGG_schmidtea to modify the call of the number of class. MODIFICATION NOT TESTED


In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn

import csv
import pickle
import seaborn as sn
import pandas as pd

from sklearn.model_selection import train_test_split

import json
from glob import glob

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import torch.nn as nn
import numpy as np
import csv

import import_ipynb

from tools.ToolBox import json_loader
from tools.ToolBox import csv_saver

from tools.CNN_Tools import VGG_Schmidtea
from tools.CNN_Tools import train
from tools.CNN_Tools import validate

from tools.Dataset_Tools import centriole_dataset
from tools.Dataset_Tools import dataset_creator
from tools.Dataset_Tools import dataset_loader
from tools.Dataset_Tools import dataset_aggregator

importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/ToolBox.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/CNN_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Dataset_Tools.ipynb


In [3]:
# Path to the model that you want to FineTune
WEIGHT_PATH = './weight/VGG_schmidtea_weight_classification.pth'

In [4]:
problem = 'regression_4'
#problem = 'classification'
n_class = 1

Loss = 1

if problem == 'classification':
    criterion = nn.CrossEntropyLoss()

    # n class expected in the new model
    nClass = 360
    
elif problem == 'regression_1':
    # Angle in degree
    def criterion(output, target):
        a = output - target
        a = (a + 180) % 360 - 180
        loss = a ** 2
        
        return loss.sum()
    
    nClass = 1
   
elif problem == 'regression_2':
    def criterion(output, target):
        loss = 2*(1-torch.cos(output - target))
        
        return loss.sum()
    nClass = 1
        

elif problem == 'regression_3' :
    def criterion(output, target):
       """ LOSS FUNCTION FOR POSITION ANGLE BASED ON MINIMUM ARC SEPARATION """
       loss = torch.mean(torch.stack([ (output-target)**2,
                                      (1-torch.abs(output-target))**2] ).min(dim=0)[0])
       return loss.sum()

    nClass = 1
    
elif problem == 'regression_4':
    criterion = nn.MSELoss()
    nClass = 1

## Model that you want to modify
model = VGG_Schmidtea(n_classes = n_class)

In [5]:
# Feature Extracting: we update only the last layer so we set the requires.grad to false
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
load_dataset = True

# Creation/Loading of the Dataset
if load_dataset == False:   
    train_loader, validation_loader = dataset_creator(path_json = './data_json/',
                                                     batch_size = 700,
                                                     n_class = n_class,
                                                     save_dataset = True)  
elif problem == 'classification':
    train_loader, validation_loader = dataset_loader(path = './data/',
                                                     train_set = 'data_train_pclassification_n360_b700_normalized.pth',
                                                     val_set = 'validation_data_pclassification_n360_b700_.pth')
    
else: 
    train_loader, validation_loader = dataset_loader(path = './data/',
                                                 train_set = 'train_data_pregression_b700_unNormalized.pth',
                                                 val_set = 'validation_data_pregression_b700_unNormalized.pth')

    

In [7]:
load_weight = False

# For feature extraction: only the last layer is updated
feature_extract = True


if torch.cuda.is_available():                                  
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)



# Weight loading
if load_weight == True:
    model.load_state_dict(torch.load(WEIGHT_PATH))
    set_parameter_requires_grad(model, feature_extract)
    model.classifier[8] = nn.Linear(4096, nClass)
    model.to(device)

else: 
    model.to(device)
    
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
    


# Oprimizer loading
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.)

print("\t", model.classifier[8])

Using PyTorch version: 1.6.0  Device: cuda
Params to learn:
	 conv_1.0.weight
	 conv_1.0.bias
	 conv_1.1.weight
	 conv_1.1.bias
	 conv_1.2.weight
	 conv_1.2.bias
	 conv_2.0.weight
	 conv_2.0.bias
	 conv_2.1.weight
	 conv_2.1.bias
	 conv_2.2.weight
	 conv_2.2.bias
	 conv_3.0.weight
	 conv_3.0.bias
	 conv_3.1.weight
	 conv_3.1.bias
	 conv_3.2.weight
	 conv_3.2.bias
	 conv_3.3.weight
	 conv_3.3.bias
	 conv_4.0.weight
	 conv_4.0.bias
	 conv_4.1.weight
	 conv_4.1.bias
	 conv_4.2.weight
	 conv_4.2.bias
	 conv_4.3.weight
	 conv_4.3.bias
	 conv_5.0.weight
	 conv_5.0.bias
	 conv_5.1.weight
	 conv_5.1.bias
	 conv_5.2.weight
	 conv_5.2.bias
	 conv_5.3.weight
	 conv_5.3.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.5.weight
	 classifier.5.bias
	 classifier.7.weight
	 classifier.7.bias
	 classifier.8.weight
	 classifier.8.bias
	 Linear(in_features=4096, out_features=1, bias=True)


In [8]:
print(f"Problem: {problem}")
print(f"Number of Class: {nClass}")
print(f"Criterion : {criterion}")
print(f"Device: {device}")

Problem: regression_4
Number of Class: 1
Criterion : MSELoss()
Device: cuda


In [9]:
%%time

# Define the number of epochs to train the model
epochs = 3
performed_epochs = 0


# Let's Train!!
for epoch in range(1, epochs + 1):
    # Instantiate variable to log loss and accuracy
    acct, losst, accv, lossv,cmt, cmv  = [], [], [], [], [], []
    list_target = []
    
    train(model, losst, acct, cmt, train_loader, device, problem, criterion, optimizer, epoch, nClass)    
    validate(model, lossv, accv, cmv, validation_loader, device, problem, criterion, nClass, list_target)
    
    if epoch%100 == 0:
        torch.save(model.state_dict(),('./weight/VGG_schmidtea_weigth_epoch_classification_n360_' + str(epoch+performed_epochs) + '_TOTO.pth'))
    
    """
    csv_saver(accv , './metrics/accv.csv')
    csv_saver(lossv, './metrics/lossv.csv')
    csv_saver(acct , './metrics/acct.csv')
    csv_saver(losst, './metrics/losst.csv')
    csv_saver(cmt  , './metrics/cmt.csv')
    csv_saver(cmv  , './metrics/cmv.csv')
    """
    
    
#torch.save(model.state_dict(),('VGG_schmidtea_weigth_epoch_final.pth'))

Train Epoch: 1 [0/559746 (0%)]	Loss: 48751.531250


RuntimeError: CUDA out of memory. Tried to allocate 44.00 MiB (GPU 0; 7.80 GiB total capacity; 6.67 GiB already allocated; 40.06 MiB free; 6.71 GiB reserved in total by PyTorch)

In [10]:
x_list = []
y_list = []
toto = 0
for i in list_target:
    for j in i[0]:
        x_list.append(torch.flatten(j).cpu().detach().numpy())
    for j in i[1]:
        y_list.append(j.cpu().detach().numpy())
        
            
print(len(list_target))

4


In [11]:
print("nulber of list in i", len(list_target))
print("nulber of list in j", len(list_target[0]))
print("nulber of list in j", len(list_target[0][0]))


len(x_list)
len(y_list)


nulber of list in i 4
nulber of list in j 2
nulber of list in j 700


2800

In [12]:
print(int(y_list[3]))
print(int(x_list[3]))

ValueError: cannot convert float NaN to integer

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x_list, y_list)
plt.show()

In [ ]:
for idx, batch in enumerate(validation_loader):
    if idx == 131:
        print(len(batch['image']), batch['image'][362])
        print((batch['angle'][350:370]))

##### 

# ADD CONFUSION MATRIX IN REGISTERED METRICS

In CNN_TOOLS , add conf_matrix as arguments and add the arguments here too

In [ ]:
confusion_matrix = torch.zeros(nClass, nClass)

#model.load_state_dict(torch.load('./weight/VGG_schmidtea_weigth_epoch_1200.pth'))

for batch_idx, batch in enumerate(validation_loader):
    # Copy data to GPU if needed
    img = batch['image'].float().to(device)
    angle = batch['angle'].long().to(device)

    with torch.no_grad():   
        output = model(img)  

    # get the index of the max log-probability
    pred_angle = output.max(1)[1]

    for a, p in zip(angle.view(-1), pred_angle.view(-1)):
        confusion_matrix[a.long(), p.long()] +=1
            
# Data in %
col_sum = confusion_matrix.numpy().sum(axis=1)
col_sum = np.reshape(col_sum, [n_class, -1])

confusion_matrix = confusion_matrix / col_sum
    
df_cm = pd.DataFrame(confusion_matrix.numpy())

plt.figure(figsize = (12,8))
sn.heatmap(df_cm)

In [ ]:
for i in range(72):
    print("Number of {} : {}".format(i, angle_db.count(i)))

=======================================

In [ ]:
class My_loss_function(nn.Module):
    
    def __init__(self):
        super(My_loss_function, self).__init__()
        
    def forward(self, output, labels):
        labels = labels.view(-1)

In [ ]:
#dataset_train = centriole_dataset(img_db = x_train, angle_db = y_train)
import matplotlib.pyplot as plt

fig = plt.figure()

for i in range(len(dataset_train)):
    sample = dataset_train[i+10]

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('angle {}°'.format(int(sample['angle'])))
    ax.axis('off')
    show_centriole(np.array(sample['image'], dtype = 'uint8'))

    if i == 3:
        plt.show()
        break


In [ ]:
dataset_test = centriole_dataset(img_db = x_test, angle_db = y_test)


fig = plt.figure()

for i in range(len(dataset_test)):
    sample = dataset_test[i+10]

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('angle {}°'.format(int(sample['angle'])))
    ax.axis('off')
    show_centriole(np.array(sample['image'], dtype = 'uint8'))

    if i == 3:
        plt.show()
        break
